In [ ]:
import sys
sys.path.append('..')
import importlib
import neural_policies
import main
import game_utils
import policy_gradient

import torch
import yaml
import numpy as np
import pyspiel
from types import SimpleNamespace

importlib.reload(main)
importlib.reload(neural_policies)
importlib.reload(game_utils)
importlib.reload(policy_gradient)

seed = 1
torch.random.manual_seed(seed)
np.random.seed(seed)

In [ ]:
# game = game_utils.create_random_normal_game(5)
# game = main.create_game(SimpleNamespace(game="rps"))
game = main.create_game(SimpleNamespace(game="kuhn_poker"))
# nn_config_file = "../config/linear_mlp.yaml"
nn_config_file = "../config/kuhn_mlp.yaml"
# nn_config_file = "../config/kuhn_big.yaml"
reinforce_config_file = "../config/reinforce_train.yaml"
nn_config = yaml.safe_load(open(nn_config_file, "r"))
reinforce_config = yaml.safe_load(open(reinforce_config_file, "r"))

In [ ]:
nash_policy, nash_value = main.nash_equilibrium_policy_and_value(game)
print("nash value", nash_value)
nn_player = 0
nn_policy = neural_policies.create_policy_net(game, nn_config)
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)

# ignore from here

In [ ]:
# s = game_utils.all_game_states(game, 0)[0]
# s.legal_actions_mask()
# s.information_state_tensor()

In [ ]:
from policy_gradient import PolicyGradientHypernetTrainer

# hypernet = main.train_hypernet_actionoutput(game, nn_config, nn_player, nn_policy, "../config/hypernet.yaml", "../config/hypernet_reinforce_train.yaml")

hypernet_config = "../config/hypernet.yaml"
hypernet_config = yaml.safe_load(open(hypernet_config, "r"))
hypernet_train_config = "../config/hypernet_reinforce_train.yaml"
hypernet_train_config = yaml.safe_load(open(hypernet_train_config, "r"))

mlp = neural_policies.create_policy_net(game, nn_config)
# hypernet = neural_policies.create_hypernet_nn_output(game, mlp, hypernet_config)
hypernet = neural_policies.create_hypernet_actionoutput(game, mlp, hypernet_config)
trainer = PolicyGradientHypernetTrainer(game, hypernet, hypernet_train_config)

In [ ]:
trainer.train_best_response(nn_config, 1 - nn_player)

In [ ]:
nn_policy = neural_policies.create_policy_net(game, nn_config)
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)
main.tabular_br(game, nn_player, nn_tab_policy)
main.policy_gradient_br(
    game, nn_config, reinforce_config, nn_player, nn_policy, nn_tab_policy
)
main.hypernet_br(game, hypernet, nn_player, nn_policy, nn_tab_policy)

lst_policy = None

In [ ]:
# print(game.row_utilities())

new_input_nn = neural_policies.create_policy_net(game, nn_config)
input_probs = neural_policies.nn_to_tabular_policy(game, new_input_nn, nn_player).action_probability_array
# input_probs = neural_policies.get_nn_probs(new_input_nn, game.new_initial_state(), nn_player)
# print("input probs", input_probs)

playable_states = game_utils.all_game_states(game, 1-nn_player)
state = playable_states[0]

# print(neural_policies.get_hypernet_probs(hypernet, new_input_nn, state, 1-nn_player))
# print(hypernet(new_input_nn, torch.tensor(state.information_state_tensor(1-nn_player))))
nn_response = neural_policies.nn_to_tabular_policy(game, hypernet, 1-nn_player, new_input_nn)
# print("nn response", nn_response.action_probability_array)
br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1 - nn_player)
print(lst_policy)
if lst_policy is not None:
    cur_policy = br.action_probability_array
    # find rows that differ
    diff = np.where(cur_policy != lst_policy)
    print("diff", diff)
lst_policy = br.action_probability_array
print(br.action_probability_array)

# neural_br_policy = neural_policies.create_policy_net(game, nn_config)
# trainer = policy_gradient.PolicyGradientTrainer(
#     game, neural_br_policy, train_params=reinforce_config
# )
# trainer.train_best_response(nn_policy, 1-nn_player)
# print(neural_br_policy(torch.tensor(game.new_initial_state().information_state_tensor(1-nn_player))))
# print(neural_policies.get_nn_probs(neural_br_policy, game.new_initial_state(), 1-nn_player))

In [ ]:
from pprint import pprint
# pprint(list(hypernet._model.get_weights()))
mm = hypernet._model
[l.bias for l in mm.layers]
# mm.layers[-1].weight

In [ ]:
import input_net_data
importlib.reload(input_net_data)
# input_nets = input_net_data.read_all_nn(game, "../nn_inputs/kuhn")
input_nets = input_net_data.read_all_nn(game, "../nn_inputs/kuhn-0-big")
# input_nets = input_net_data.read_all_nn(game, "../nn_inputs/rps")

In [ ]:
ind = -1
nn_player

In [ ]:
# nn_policy = neural_policies.create_policy_net(game, nn_config)
ind = ind + 1
# import random
# ind = random.choice([i for i in range(5)])
ind = ind % 5
print("ind", ind)
# print(input_nets[ind].get_weights())
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, input_nets[ind], 0)
# br_pol = main.tabular_br(game, nn_player, nn_tab_policy)
print(nn_tab_policy.action_probability_array)

# print(nn_tab_policy.action_probability_array)

br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1)
print(br.action_probability_array)
combined_policy = main.combine_tabular_policies(game, nn_tab_policy, br)
print("value of table br policy", main.policy_value(game, combined_policy))

nn_br = neural_policies.nn_to_tabular_policy(game, hypernet, 1-nn_player, input_nets[ind])
print(nn_br.action_probability_array)
diff = np.abs(br.action_probability_array - nn_br.action_probability_array).sum()
print(diff)
combined_policy = main.combine_tabular_policies(game, nn_tab_policy, nn_br)
print("value of table br policy", main.policy_value(game, combined_policy))

In [ ]:
for s in combined_policy.states:
    print(s.information_state_tensor(), s.information_state_string())

In [ ]:
from nfn.common import state_dict_to_tensors
state_dict_to_tensors(input_nets[0].state_dict())[0][0].shape

In [ ]:
nn = input_nets[0]
w1 = list(nn.parameters())[0]

# relevant from here, next two cells

In [ ]:
import policy_gradient
importlib.reload(policy_gradient)
importlib.reload(neural_policies)

hypernet_config = "../config/hypernet.yaml"
hypernet_config = yaml.safe_load(open(hypernet_config, "r"))
hypernet_train_config = "../config/hypernet_reinforce_train.yaml"
hypernet_train_config = yaml.safe_load(open(hypernet_train_config, "r"))

mlp = neural_policies.create_policy_net(game, nn_config)
hypernet_nn_output = neural_policies.create_hypernet_nn_output(game, mlp, hypernet_config)
trainer = policy_gradient.PolicyGradientHypernetTrainer(game, hypernet_nn_output, hypernet_train_config)

In [ ]:
policy_nets = trainer.train_simultaneous_br(nn_config)
# trainer.train_best_response(policy_nets, 0)

In [ ]:
ind = -1
nn_config
game
input_nets[0]

In [ ]:
# nn_policy = neural_policies.create_policy_net(game, nn_config)
ind = ind + 1
nn_player = 1
nn_policy = input_nets[ind]
nn_tab_policy = neural_policies.nn_to_tabular_policy(game, nn_policy, nn_player)
# br_pol = main.tabular_br(game, 1-nn_player, nn_tab_policy)
print(nn_tab_policy.action_probability_array)

# main.policy_gradient_br(
#     game, nn_config, reinforce_config, nn_player, nn_policy, nn_tab_policy
# )

br = main.compute_best_response_tabular_policy(game, nn_tab_policy, 1-nn_player)
print(br.action_probability_array)
policies = [nn_tab_policy, br] if nn_player == 0 else [br, nn_tab_policy]
combined_policy = main.combine_tabular_policies(game, *policies)
print("value of table br policy", main.policy_value(game, combined_policy))

nn_br = neural_policies.nn_to_tabular_policy(game, policy_nets[1-nn_player], 1-nn_player, nn_policy)
print(nn_br.action_probability_array)
policies = [nn_tab_policy, nn_br] if nn_player == 0 else [nn_br, nn_tab_policy]
combined_policy = main.combine_tabular_policies(game, *policies)
print("value of hypernet br policy", main.policy_value(game, combined_policy))

In [ ]:
pnout = policy_nets[1].model_output(input_nets[ind])
[print(x) for x in pnout.parameters()]

In [ ]:
pnout
game.information_state_tensor_shape()

In [ ]:
g2 = pyspiel.load_game("leduc_poker")
g2.information_state_tensor_shape()
s = game.new_initial_state()
s.information_state_tensor(1)

# test setting weights

In [ ]:
import neural_policies
importlib.reload(neural_policies)
import torch.nn as nn

In [ ]:
mlp = neural_policies.create_policy_net(game, nn_config)

In [ ]:
new_weights = torch.ones_like(mlp.get_weights(), requires_grad=True)

In [ ]:
new_weights.requires_grad, new_weights.grad

In [ ]:
mlp.set_weights(new_weights)

In [ ]:
mlp.network[0].weight.grad

In [ ]:
res = mlp.forward(torch.zeros(1))

In [ ]:
loss = res.sum()
loss.backward()

In [ ]:
mlp.network[0].weight.grad

In [ ]:
new_weights.grad

In [ ]:
linear = nn.Linear(2, 2)
linear.weight.grad

In [ ]:
new_weights = torch.ones(8, requires_grad=True)
# linear.weight = nn.Parameter(new_weights)
linear.weight.requires_grad = False
linear.weight.copy_(new_weights[:4].view(linear.weight.shape))

In [ ]:
inp = torch.ones(2, requires_grad=True)
loss = linear(inp).sum()

In [ ]:
loss.backward()

In [ ]:
inp.grad
linear.weight.grad

In [ ]:
new_weights.grad